# 실전 예측 분석 모델링

## **6장. 선형 회귀와 이웃 모델들**

$y_i = b_0 + b_1 x_{i1} + b_2 x_{i2} +  + b_P x_{iP} + e_i$
* 일반적인 선형 회귀에서는 최소 편향값을 가진 변수를 추정하고자 하지만, 능형회귀(ridge regression), 라소, 엘라스틱 넷에서는 낮은 분산을 가진 추정값을 찾는다.
    * (최소 편향 -> 과적합), (낮은 분산 -> 과소적합)
* 위의 식 처럼 변수들이 선형관계일 때의 장점은 계수의 표준 오차를 구해 모델 잔차의 분포에 대해 어떤 가정을 할 수 있도록 한다는 것이다.
이런 표준편차를 통해 모델의 각 예측 변수의 통계적 유의성을 판단할 수 있다.

### 6.1 사례연구 : 구조적 정량 활성 관계 모델링
* 우도를 확인했을 때, 최솟값 0.7, 최댓값 3.8로 이 예측 변수가 오른쪽으로 치우쳐 있다는 것을 확인하여 예측 변수에 box-cox변환 적용.
* 이렇게 변형된 예측 변수 간 상관관계가 여전히 유의한지를 알기 위해서는 변환된 전체 예측 변수에 주성분 분석(PCA)을 적용해 각 성분에 사용된 분산 비율로 판단해야 함.
    * '스크리 플롯' 확인 결과 분산의 13%이상인 시점에서는 변동량이 급격하게 떨어지는 것을 확인. <br>
    이를 통해 원 공간의 차원에 비해 실제로는 훨씬 적은 차원의 정보만 포함한다는 것을 알 수 있다. <br>
    이는 예측 변수 간 많은 공선성으로 인해 일어난다.
    
### 6.2 선형회귀
=> 보통 최소 제곱 선형 회귀의 목적은 관측값과 예측 응답값 간의 오차 제곱합(SSE)을 최소화하는 평면을 찾는 것이다.

#### 다중 선형회귀에서의 몇가지 문제점 해결방법
1) 예측 변수 수가 관측 수보다 큰 경우 => 예측 변수의 수를 제거
* 선형회귀상에서 다중 공선성을 판단할 때는 '분산 팽창 인수(variance inflation factor)' 를 사용한다.
    * 이 통곗값은 각 예측 변수별로 해당 예측 변수와 모든 다른 예측 변수 간의 상관관계를 구한 것이다
* 전처리 후에도 예측 변수 수가 관측 수보다 크다면, PCA처리하는 방법 있음.
    * 또는, PLS나 릿지 회귀, 라소, 엘라스틱 넷 같이 인수 추정을 감소하는 방식을 사용한 회귀와 동시에 차원을 축소하는 방식.

2) 결과로 나온 변수들이 선형관계일 경우 => 예측 변수와 응답 변수 간의 관계가 선형인지, 아닌지를 파악
* 예측 대 잔차 그래프 활용
* 예측 변수와 응답 변수 간 바로 파악되는 비선형 관계가 존재할 경우, 예측 변수의 2차, 3차 변수를 회귀에 추가하는 식으로 변형.
* 이런 관계를 파악할 수 없거나, 매우 비선형적이면 신경망모델, SVM, k-최근접 이웃 모델 등 보다 복잡한 방법을 사용하는 것이 효율적

3) 추세로부터 떨어져 있는 관측값을 좇아가려는 경향 => SSE를 최소화 하려면, 이런 특이한 관측값을 잘 수용할 수 있는 인수 추정값을 반영해야 함
    (주요 추이에서 벗어나 있는 관측값은 기하급수적으로 큰 잔차를 갖게 됨)
* SSE 대신 큰 이상값에 대해 덜 민감한 다른 지표를 사용. (ex. 오차 절댓값의 합을 최소화하는 추정인수)
* 후버 함수

4) 예측 변수의 상관관계가 클 경우
* 수동으로 두 변수 중 하나를 제거하는 식으로 조정
* 예측 변수가 많은 경우네는 수동으로 제거하기 어려우므로 이런 공선성에 영향을 받지 않을 수 있는 모델을 사용하는 것이 낫다.

### 6.3 부분 최소 제곱
**PCA를 사용할 때의 문제점** <br>
=> 참고로 여기서 PCA를 사용해 예측 변수를 전처리하는 것은 주성분 회귀 principle component regression, PCR 로 알려져 있다.
* 기존 변수의 선형 조합으로 새로운 예측 변수를 만들어 내게 되므로 이 새 변수를 해석하는 데 어려움이 있을 수 있다.
    * 즉, PCA를 사용해서 X1 과 X2 로 새로운 변수 P를 만들었는데 이 P라는 변수는 종속변수(Y값)에 대한 아무런 예측 정보를 갖고 있지 않을 수 있다.

**PCA의 문제점을 해결하기 위한 방안으로 PLS 추천**
* 상관관계가 있는 예측 변수가 있고, 선형 회귀 형태의 해답이 필요한 경우 PLS를 추천
    * PLS는 인수가 비선형적인 모델을 선형 형태로 만드는 헤르만 볼드의 비선형 재귀 부분 최소 제곱(nonlinear iterative partial least squares, NIPALS) 알고리즘으로부터 나왔다.
    * NIPALS알고리즘은 재귀적으로 응답 변수와 높은 상관관계를 갖는 예측 변수 내부, 또는 잠재하는 관계를 찾아낼 수 있다.
* PLS를 실행하기 전에 특히 예측 변수가 서로 다른 단위일 경우에는 예측 변수에 중심화 및 척도화를 실행한다.

**PCA, PLS 차이점**
* PCA 가 예측 변수 공간의 변동성을 최대로 하는 값을 선택한다면,
* PLS는 응답 변수의 공분산을 최대로 하는 값을 고른다.
    * 즉, PLS는 예측 변수의 분산을 최대로 하는 성분을 찾는다 .이때 이 성분은 응답변수와 상관관계가 가장 커야 한다 .
* PLS는 '지도'차원 축소과정, PCA는 '비지도' 과정
* PCA 방향과 PLS방향은 거의 직각으로, 최적의 차원 축소 방향과 예측 변수 공간의 최대 변량과는 관계가 없다는 것을 나타낸다.
* 실제로 PCA는 PLS와 거의 같은 예측력을 갖는 모델을 생성한다. <br>
경험상, PCA를 통해 남은 성분 수는 PLS가 가진 성분 수보다 크거나 같다.

**PCA 와 PLS 실험결과 비교**
* PCA 적용 결과, 35개의 성분일때 RMSE = 0.731 
* PLS로 적용 결과, 10개의 성분일때 RMSE = 0.682 로 지도 차원 축소 시에는 비지도 차원 축소 시보다 확실히 적은 성분을 사용해 최소의 RMSE를 찾아낸다.
* PLS를 사용할 경우 PCA보다 훨씬 적은 수의 성분을 사용하므로 보다 단순한 모델을 만들 수 있다.

### 6.4 벌점 모델 (Lidge, Lasso, 엘라스틱 넷)
편향적인 모델로 인해 발생하는 공선성 문제를 해결하면, 총 MSE가 좋은 회귀 모델을 구축할 수 있다. <br>
편향적 회귀 모델을 만드는 한 가지 방법은 오차 제곱 합에 벌점을 부과하는 것이다 <br>
=> 즉, 계수의 크기를 줄이는 것이 목적 (정규화)

#### 릿지 회귀
=> 영향을 거의 미치지 않는 특성에 대하여 0에 가까운 가중치를 준다.

* 모델이 데이터에 과적합을 일으키거나 공선성 문제가 발생하는 경우, 선형 회귀 인수 추정값은 급격히 커진다.
    * 이런 경우, 추정값의 크기를 조정해 SSE를 감소시키고자 할 것이다.
    * 추정값이 커졌을 경우, SSE에 벌점을 부과하는 방식으로 인수 추정값을 조절(또는 정규화) 할 수 있다. 
    * 릿지 회귀 에서는 회귀 인수 제곱합에 벌점을 부과한다.
* 다중 공선성 방지에 가장 많이 쓰임.

#### 라쏘 회귀
=> 특성값의 계수가 매우 낮다면 0으로 수렴하게 하여 특성을 지워버린다 .특성이 모델에 미치는 영향을 0으로 만든다는 것은 bias를 증가시켜 overfitting을 방지한다는 의미.

* 라소 모델에서는 동시에 정규화를 사용해 모델 성능을 향상시키고 특징 선택을 할 수 있도록 한다.
* 변수를 자동으로 채택할 수 있어 일반적으로 많은 변수를 다룰 때 활용.

#### 엘라스틱 넷
=> 라쏘회귀와 릿지회귀의 최적화 지점이 서로 다르기 때문에 두 정규화항을 합쳐서 규제정도를 조절해준다
